In [1]:
from bs4 import BeautifulSoup as bsoup
import urllib.robotparser
import requests
import time
import pandas as pd
import re
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [2]:
# daily revenue for movie industry, contains data for the top grossing movie by day with associated revenue
# test run: goes to url and pulls all data for 1 month

url = "https://www.boxofficemojo.com/daily/2019/?interval=january&sort=date&sortDir=asc&view=month&ref_=bo_di__resort#table"
page = requests.get(url)
soup = bsoup(page.text, 'lxml')

all_divs = soup.main.find_all("div", id="table")
all_tables = all_divs[0].find_all("table")
all_trs = all_tables[0].find_all("tr")

for i in range(1, len(all_trs)):
    print(f"Entry row {i} = ")
    
    all_tds = all_trs[i].find_all("td")
    entry = []
    
    for j in range(0, len(all_tds)):
        entry.append(all_tds[j].text)       
    
    print(entry)
    

Entry row 1 = 
["Jan 1, 2019New Year's Day", 'Tuesday', '1', '$52,588,390', '-0.3%', '+23.3%', '53', 'Aquaman', '$16,377,779']
Entry row 2 = 
['Jan 2, 2019', 'Wednesday', '2', '$25,043,176', '-52.4%', '-54.9%', '53', 'Aquaman', '$7,379,476']
Entry row 3 = 
['Jan 3, 2019', 'Thursday', '3', '$22,001,708', '-12.1%', '-57.4%', '55', 'Aquaman', '$6,203,801']
Entry row 4 = 
['Jan 4, 2019', 'Friday', '4', '$38,870,140', '+76.7%', '-31.4%', '52', 'Aquaman', '$9,388,082']
Entry row 5 = 
['Jan 5, 2019', 'Saturday', '5', '$48,843,884', '+25.7%', '-16%', '52', 'Aquaman', '$13,053,690']
Entry row 6 = 
['Jan 6, 2019', 'Sunday', '6', '$31,293,715', '-35.9%', '-38.6%', '52', 'Aquaman', '$8,561,508']
Entry row 7 = 
['Jan 7, 2019', 'Monday', '7', '$9,979,794', '-68.1%', '-72.5%', '48', 'Aquaman', '$2,583,295']
Entry row 8 = 
['Jan 8, 2019', 'Tuesday', '8', '$14,364,595', '+43.9%', '-72.7%', '50', 'Aquaman', '$3,852,619']
Entry row 9 = 
['Jan 9, 2019', 'Wednesday', '9', '$9,403,436', '-34.5%', '-62.5%', 

In [3]:
# daily revenue for movie industry, contains data for the top grossing movie by day with associated revenue
# this will run through all months and the past 4 years worth of data on boxofficemojo.com to gather the revenue data
# the result is a data frame with the top grossing movie by day for the past 4 years


months = ["january","february","march","april","may","june","july","august","september", "october", "november", "december"]
years = ["2014","2015","2016","2017","2018","2019"]

column_names = ["date_full","weekday","day_number","top_10_gross_total","ytd_delta",
                "lw_delta","releases","top_release","top_gross"]

df = pd.DataFrame(columns=column_names)

print("running")

for year in years:
    for month in months:
        url_a = "https://www.boxofficemojo.com/daily/"
        url_b = year
        url_c = "/?interval="
        url_d = month
        url_e = "&sort=date&sortDir=asc&view=month&ref_=bo_di__resort#table"

        url = url_a + url_b + url_c + url_d + url_e
        page = requests.get(url)
        soup = bsoup(page.text, 'lxml')
        current_month = []

        all_divs = soup.main.find_all("div", id="table")
        all_tables = all_divs[0].find_all("table")
        all_trs = all_tables[0].find_all("tr")

        for i in range(1, len(all_trs)):
            all_tds = all_trs[i].find_all("td")
            entry = []
            for j in range(0, len(all_tds)):
                entry.append(all_tds[j].text)
            current_month.append(entry)

        df_month = pd.DataFrame(current_month, columns=column_names)
        frames = [df, df_month]
        df = pd.concat(frames)
        
        time.sleep(0.1)
        
    print(f"{year} processed...")
    
df.reset_index(inplace=True)
df.drop(["index", "ytd_delta", "lw_delta"], axis = 1, inplace = True)
print("complete")

running
2014 processed...
2015 processed...
2016 processed...
2017 processed...
2018 processed...
2019 processed...
complete


In [4]:
df.head(10)

,date_full,weekday,day_number,top_10_gross_total,releases,top_release,top_gross
0,"Jan 1, 2014New Year's Day",Wednesday,1,"$48,419,707",49,Frozen,"$8,718,939"
1,"Jan 2, 2014",Thursday,2,"$25,361,378",49,Frozen,"$5,304,617"
2,"Jan 3, 2014",Friday,3,"$42,939,384",45,Paranormal Activity: The Marked Ones,"$8,722,144"
3,"Jan 4, 2014",Saturday,4,"$49,402,611",45,Frozen,"$8,037,475"
4,"Jan 5, 2014",Sunday,5,"$26,723,321",45,Frozen,"$4,785,996"
5,"Jan 6, 2014",Monday,6,"$9,984,206",45,Frozen,"$1,728,610"
6,"Jan 7, 2014",Tuesday,7,"$12,471,587",45,The Wolf of Wall Street,"$1,955,396"
7,"Jan 8, 2014",Wednesday,8,"$9,494,589",45,The Wolf of Wall Street,"$1,643,640"
8,"Jan 9, 2014",Thursday,9,"$8,724,720",45,The Wolf of Wall Street,"$1,469,075"
9,"Jan 10, 2014",Friday,10,"$36,124,969",43,Lone Survivor,"$14,403,750"


In [5]:
# cleaning up the revenue values

df['top_10_gross_total'] = df['top_10_gross_total'].str.replace("$", "")
df['top_10_gross_total'] = df['top_10_gross_total'].str.replace(",", "")
df['top_gross'] = df['top_gross'].str.replace("$", "")
df['top_gross'] = df['top_gross'].str.replace(",", "")

df.head()

,date_full,weekday,day_number,top_10_gross_total,releases,top_release,top_gross
0,"Jan 1, 2014New Year's Day",Wednesday,1,48419707,49,Frozen,8718939
1,"Jan 2, 2014",Thursday,2,25361378,49,Frozen,5304617
2,"Jan 3, 2014",Friday,3,42939384,45,Paranormal Activity: The Marked Ones,8722144
3,"Jan 4, 2014",Saturday,4,49402611,45,Frozen,8037475
4,"Jan 5, 2014",Sunday,5,26723321,45,Frozen,4785996


In [6]:
# cleaning up the date column, adding a new columns for month, day, year, and event (holiday)

dates = df[['date_full']]
holiday = pd.DataFrame(columns = ["date", "year", "event", "month", "day"])
holiday[["date", "year", "event"]] = dates.date_full.str.split('(\d{4})', expand = True)
holiday['date'] = holiday['date'].str.replace(",", "")
holiday[["month", "day"]] = holiday.date.str.split(' ', n= 1, expand = True)
final = holiday[['month', 'day', 'year', 'event']]

final.head()

,month,day,year,event
0,Jan,1,2014,New Year's Day
1,Jan,2,2014,
2,Jan,3,2014,
3,Jan,4,2014,
4,Jan,5,2014,


In [7]:
# update date columns using new dataframe from above

df[['month', 'day', 'year', 'event']] = final[['month', 'day', 'year', 'event']]
df.drop(["date_full"], axis = 1, inplace = True)

In [8]:
# df.to_csv("movie_data_v1.csv")
df.sample(20)
df['top_10_gross_total'] = pd.to_numeric(df['top_10_gross_total'])
df['releases'] = pd.to_numeric(df['releases'])
df['top_gross'] = pd.to_numeric(df['top_gross'])
df.dtypes

weekday               object
day_number            object
top_10_gross_total     int64
releases               int64
top_release           object
top_gross              int64
month                 object
day                   object
year                  object
event                 object
dtype: object

In [9]:
# aggregate data by month, year, and release
aggdat = df.groupby(['year', 'month', 'top_release']).agg({'top_gross':'sum', 'top_10_gross_total':'sum', 'releases':'mean', 'top_release':'count'}).copy()
aggdat = aggdat.rename(columns={"top_release": "total_top_days"})


In [10]:
aggdat.head(50)
aggdat.to_pickle('final-clean-box_office_mojo.pkl')